In [155]:
# Basic Libraries

import numpy as np
import scipy as sp
import sympy as smp
from scipy.integrate import solve_ivp, odeint
import itertools
import mpmath
import opt_einsum as oe
import numba as nb
import symengine as sme
from scipy.spatial import KDTree
from scipy.sparse import coo_array

# For Plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# For Debugging and Profiling
import snoop
from cProfile import Profile
from pstats import SortKey, Stats
import pyperclip

In [156]:
def get_norm(T, g):
    """
    T should have confuguration either 'u' or 'uu
    Returned Configuration: Scalar
    """
    if T.ndim == 2:
        return np.einsum("ua,vb,ab,uv->", g, g, T, T)

    else:
        return np.einsum("ab,a,b->", g, T, T)

In [157]:
def levi_civita_symbol(dim=4):
    """
    Define Levi Cevita Tensor
    Config: uuuu
    """

    arr = np.zeros(tuple([dim for _ in range(dim)]))
    for x in itertools.permutations(tuple(range(dim))):
        mat = np.zeros((dim, dim), dtype=np.int32)
        for i, j in zip(range(dim), x):
            mat[i, j] = 1
        arr[x] = int(np.linalg.det(mat))
    return arr

In [158]:
# @snoop
def levi_civita_tensor(gk):
    #! uuuu
    epsu = levi_civita_symbol()
    det_g = smp.det(smp.Matrix(gk))
    return epsu / sme.sqrt(-(det_g))

In [159]:
def cov_metric_tensor(r, theta, a, M=1.0):
    """
    Define the metric tensor function.

    Parameters:
        a (float): Parameter 'a' in the metric.
        r (float): Parameter 'r' in the metric.
        theta (float): Parameter 'theta' in the metric.

    Returns:
        np.ndarray: The metric tensor at the given values of a, r, and theta.
        Configuration: ll
    """
    #! Correction Added Mass term in the symbols
    g = np.zeros((4, 4), dtype=type(r))

    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    g[0, 0] = -(1.0 - 2.0 * M * r / Sigma)
    g[1, 1] = Sigma / Delta
    g[2, 2] = Sigma
    g[3, 3] = (
        sme.sin(theta) ** 2
        * ((r**2 + a**2) ** 2 - Delta * a**2 * sme.sin(theta) ** 2)
        / Sigma
    )
    g[0, 3] = -2.0 * a * M * r * sme.sin(theta) ** 2 / Sigma
    g[3, 0] = g[0, 3]

    # return g.astype(float)
    return g

In [160]:
# @nb.njit
def Bcov_metric_tensor(r, theta, a, M):
    return np.array(
        [
            [
                -(
                    (a**2 - 2 * M * r + r**2 - a**2 * sme.sin(theta) ** 2)
                    / (r**2 + a**2 * sme.cos(theta) ** 2)
                ),
                0,
                0,
                (-2 * a * M * r * sme.sin(theta) ** 2)
                / (r**2 + a**2 * sme.cos(theta) ** 2),
            ],
            [0, (r**2 + a**2 * sme.cos(theta) ** 2) / (a**2 - 2 * M * r + r**2), 0, 0],
            [0, 0, r**2 + a**2 * sme.cos(theta) ** 2, 0],
            [
                (-2 * a * M * r * sme.sin(theta) ** 2)
                / (r**2 + a**2 * sme.cos(theta) ** 2),
                0,
                0,
                (
                    sme.sin(theta) ** 2
                    * (
                        (a**2 + r**2) ** 2
                        - a**2 * (a**2 + r * (-2 * M + r)) * sme.sin(theta) ** 2
                    )
                )
                / (r**2 + a**2 * sme.cos(theta) ** 2),
            ],
        ]
    )

In [161]:
def cot_metric_tensor(r, theta, a, M=1.0):
    """
    Define the metric tensor function.

    Parameters:
        a (float): Parameter 'a' in the metric.
        r (float): Parameter 'r' in the metric.
        theta (float): Parameter 'theta' in the metric.

    Returns:
        np.ndarray: The metric tensor at the given values of a, r, and theta.
        Configuration: uu
    """
    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    denom = Delta * sme.sin(theta) ** 2

    g = np.zeros((4, 4), dtype=type(r))
    g[0, 0] = (
        -(
            sme.sin(theta) ** 2
            * ((r**2 + a**2) ** 2 - Delta * a**2 * sme.sin(theta) ** 2)
            / Sigma
        )
        / denom
    )

    g[1, 1] = Delta / Sigma

    g[2, 2] = 1.0 / Sigma

    # g[3, 3] = -(-(1.0 - 2.0 * r / Sigma)) / denom
    #! Corrected the g[3,3] term below
    g[3, 3] = (Delta - a**2 * sme.sin(theta) ** 2) / (denom * Sigma)

    g[0, 3] = (-2.0 * a * M * r * sme.sin(theta) ** 2 / Sigma) / denom

    g[3, 0] = g[0, 3]

    return g
    # return g

In [162]:
# @nb.njit
def Bcot_metric_tensor(r, theta, a, M):
    return np.array(
        [
            [
                -(
                    (
                        a**4
                        + 2 * r**4
                        + a**2 * r * (2 * M + 3 * r)
                        + a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                    )
                    / (
                        (a**2 + r * (-2 * M + r))
                        * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta))
                    )
                ),
                0,
                0,
                (-4 * a * M * r)
                / (
                    (a**2 + r * (-2 * M + r))
                    * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta))
                ),
            ],
            [0, (a**2 - 2 * M * r + r**2) / (r**2 + a**2 * sme.cos(theta) ** 2), 0, 0],
            [0, 0, (r**2 + a**2 * sme.cos(theta) ** 2) ** (-1), 0],
            [
                (-4 * a * M * r)
                / (
                    (a**2 + r * (-2 * M + r))
                    * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta))
                ),
                0,
                0,
                (
                    (a**2 + 2 * r * (-2 * M + r) + a**2 * sme.cos(2 * theta))
                    * (1 / (sme.sin(theta))) ** 2
                )
                / (
                    (a**2 + r * (-2 * M + r))
                    * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta))
                ),
            ],
        ]
    )

In [163]:
def kerr_christoffel(r, theta, a, M=1.0):
    """
    Function to give the christoffel symbols for kerr metric.
    The christoffel symbols are given as \Gamma ^i _{jk}

    From Reference Paper, Appendix
    Config: ull
    """

    cs = np.zeros((4, 4, 4), dtype=type(r))

    # Definitions
    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    cs[3, 0, 1] = M * (2 * r**2 - Sigma) / (Delta * Sigma**2) * a
    cs[0, 0, 1] = (
        M * (r**2 + a**2) * (r**2 - a**2 * sme.cos(theta) ** 2) / (Sigma**2 * Delta)
    )

    cs[3, 0, 2] = -2 * M * a * r * (sme.cos(theta) / sme.sin(theta)) / (Sigma**2)
    cs[0, 0, 2] = -2 * M * a**2 * r * sme.sin(theta) * sme.cos(theta) / Sigma**2

    cs[0, 1, 3] = (
        -M
        * a
        * (2 * r**2 * (r**2 + a**2) + Sigma * (r**2 - a**2))
        * sme.sin(theta) ** 2
        / (Delta * Sigma**2)
    )

    cs[3, 1, 3] = (
        r * Sigma * (Sigma - 2 * M * r)
        - M * a**2 * (2 * r**2 - Sigma) * sme.sin(theta) ** 2
    ) / (Delta * Sigma**2)

    cs[0, 2, 3] = M * a**3 * r * sme.sin(theta) ** 2 * sme.sin(2 * theta) / Sigma**2

    cs[3, 2, 3] = (
        (sme.cos(theta) / sme.sin(theta))
        * (Sigma**2 + 2 * M * a**2 * r * sme.sin(theta) ** 2)
        / Sigma**2
    )

    cs[1, 0, 0] = M * Delta * (2 * r**2 - Sigma) / Sigma**3

    cs[1, 0, 3] = -cs[1, 0, 0] * a * sme.sin(theta) ** 2

    cs[2, 0, 0] = -M * a * r * sme.sin(2 * theta) / Sigma**3 * a
    cs[2, 0, 3] = (
        2.0 * M * r * a * (r**2 + a**2) * sme.sin(theta) * sme.cos(theta) / Sigma**3
    )

    cs[1, 1, 1] = r / Sigma + (M - r) / Delta

    cs[1, 2, 2] = -r * Delta / Sigma
    cs[2, 1, 2] = -cs[1, 2, 2] / Delta

    cs[1, 1, 2] = cs[2, 2, 2] = -(a**2) * sme.sin(2 * theta) / (2 * Sigma)
    cs[2, 1, 1] = -cs[1, 1, 2] / Delta

    cs[1, 3, 3] = (
        -Delta
        * (r * Sigma**2 - M * a**2 * (2 * r**2 - Sigma) * sme.sin(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**3
    )

    cs[2, 3, 3] = (
        -(Delta * Sigma**2 + 2 * M * r * (r**2 + a**2) ** 2)
        * sme.sin(2 * theta)
        / (2 * Sigma**3)
    )

    cs[0, 1, 0] = cs[0, 0, 1]
    cs[0, 2, 0] = cs[0, 0, 2]
    cs[0, 3, 1] = cs[0, 1, 3]
    cs[0, 3, 2] = cs[0, 2, 3]

    cs[1, 3, 0] = cs[1, 0, 3]
    cs[1, 2, 1] = cs[1, 1, 2]

    cs[2, 3, 0] = cs[2, 0, 3]
    cs[2, 2, 1] = cs[2, 1, 2]

    cs[3, 1, 0] = cs[3, 0, 1]
    cs[3, 2, 0] = cs[3, 0, 2]
    cs[3, 3, 1] = cs[3, 1, 3]
    cs[3, 3, 2] = cs[3, 2, 3]

    # return symmetrize(arr=cs)+np.zeros((4,4,4))
    return cs + np.zeros((4, 4, 4))
    # return cs

In [164]:
# @nb.njit
def Bkerr_christoffel(r, theta, a, M):
    return np.array(
        [
            [
                [
                    0,
                    (
                        -2
                        * M
                        * (a**2 + r**2)
                        * (a**2 - 2 * r**2 + a**2 * sme.cos(2 * theta))
                    )
                    / (
                        (a**2 + r * (-2 * M + r))
                        * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2
                    ),
                    (-4 * a**2 * M * r * sme.sin(2 * theta))
                    / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2,
                    0,
                ],
                [
                    (
                        -2
                        * M
                        * (a**2 + r**2)
                        * (a**2 - 2 * r**2 + a**2 * sme.cos(2 * theta))
                    )
                    / (
                        (a**2 + r * (-2 * M + r))
                        * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2
                    ),
                    0,
                    0,
                    (
                        2
                        * a
                        * M
                        * (
                            a**4
                            - 3 * a**2 * r**2
                            - 6 * r**4
                            + a**2 * (a**2 - r**2) * sme.cos(2 * theta)
                        )
                        * sme.sin(theta) ** 2
                    )
                    / (
                        (a**2 + r * (-2 * M + r))
                        * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2
                    ),
                ],
                [
                    (-4 * a**2 * M * r * sme.sin(2 * theta))
                    / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2,
                    0,
                    0,
                    (8 * a**3 * M * r * sme.cos(theta) * sme.sin(theta) ** 3)
                    / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2,
                ],
                [
                    0,
                    (
                        2
                        * a
                        * M
                        * (
                            a**4
                            - 3 * a**2 * r**2
                            - 6 * r**4
                            + a**2 * (a**2 - r**2) * sme.cos(2 * theta)
                        )
                        * sme.sin(theta) ** 2
                    )
                    / (
                        (a**2 + r * (-2 * M + r))
                        * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2
                    ),
                    (8 * a**3 * M * r * sme.cos(theta) * sme.sin(theta) ** 3)
                    / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2,
                    0,
                ],
            ],
            [
                [
                    -1
                    / 2
                    * (
                        M
                        * (a**2 + r * (-2 * M + r))
                        * (a**2 - 2 * r**2 + a**2 * sme.cos(2 * theta))
                    )
                    / (r**2 + a**2 * sme.cos(theta) ** 2) ** 3,
                    0,
                    0,
                    (
                        a
                        * M
                        * (a**2 + r * (-2 * M + r))
                        * (-(r**2) + a**2 * sme.cos(theta) ** 2)
                        * sme.sin(theta) ** 2
                    )
                    / (r**2 + a**2 * sme.cos(theta) ** 2) ** 3,
                ],
                [
                    0,
                    (r * (a**2 - M * r) + a**2 * (M - r) * sme.cos(theta) ** 2)
                    / ((a**2 + r * (-2 * M + r)) * (r**2 + a**2 * sme.cos(theta) ** 2)),
                    -(
                        (a**2 * sme.cos(theta) * sme.sin(theta))
                        / (r**2 + a**2 * sme.cos(theta) ** 2)
                    ),
                    0,
                ],
                [
                    0,
                    -(
                        (a**2 * sme.cos(theta) * sme.sin(theta))
                        / (r**2 + a**2 * sme.cos(theta) ** 2)
                    ),
                    -(
                        (r * (a**2 + r * (-2 * M + r)))
                        / (r**2 + a**2 * sme.cos(theta) ** 2)
                    ),
                    0,
                ],
                [
                    (
                        a
                        * M
                        * (a**2 + r * (-2 * M + r))
                        * (-(r**2) + a**2 * sme.cos(theta) ** 2)
                        * sme.sin(theta) ** 2
                    )
                    / (r**2 + a**2 * sme.cos(theta) ** 2) ** 3,
                    0,
                    0,
                    -(
                        (
                            (a**2 + r * (-2 * M + r))
                            * sme.sin(theta) ** 2
                            * (
                                sme.cos(theta) ** 2
                                * (
                                    2 * a**2 * r * (a**2 + r**2)
                                    + a**4 * (M - r) * sme.sin(theta) ** 2
                                )
                                + r
                                * (
                                    -(a**4)
                                    + r**4
                                    + a**2 * (a**2 - M * r) * sme.sin(theta) ** 2
                                )
                            )
                        )
                        / (r**2 + a**2 * sme.cos(theta) ** 2) ** 3
                    ),
                ],
            ],
            [
                [
                    -(
                        (a**2 * M * r * sme.sin(2 * theta))
                        / (r**2 + a**2 * sme.cos(theta) ** 2) ** 3
                    ),
                    0,
                    0,
                    (a * M * r * (a**2 + r**2) * sme.sin(2 * theta))
                    / (r**2 + a**2 * sme.cos(theta) ** 2) ** 3,
                ],
                [
                    0,
                    (a**2 * sme.cos(theta) * sme.sin(theta))
                    / ((a**2 + r * (-2 * M + r)) * (r**2 + a**2 * sme.cos(theta) ** 2)),
                    r / (r**2 + a**2 * sme.cos(theta) ** 2),
                    0,
                ],
                [
                    0,
                    r / (r**2 + a**2 * sme.cos(theta) ** 2),
                    -(
                        (a**2 * sme.cos(theta) * sme.sin(theta))
                        / (r**2 + a**2 * sme.cos(theta) ** 2)
                    ),
                    0,
                ],
                [
                    (a * M * r * (a**2 + r**2) * sme.sin(2 * theta))
                    / (r**2 + a**2 * sme.cos(theta) ** 2) ** 3,
                    0,
                    0,
                    -1
                    / 16
                    * (
                        (
                            3 * a**6
                            + 10 * a**4 * M * r
                            + 11 * a**4 * r**2
                            + 16 * a**2 * M * r**3
                            + 16 * a**2 * r**4
                            + 8 * r**6
                            + 4
                            * a**2
                            * (a**2 + 2 * r**2)
                            * (a**2 + r * (-2 * M + r))
                            * sme.cos(2 * theta)
                            + a**4 * (a**2 - 2 * M * r + r**2) * sme.cos(4 * theta)
                        )
                        * sme.sin(2 * theta)
                    )
                    / (r**2 + a**2 * sme.cos(theta) ** 2) ** 3,
                ],
            ],
            [
                [
                    0,
                    (-2 * a * M * (a**2 - 2 * r**2 + a**2 * sme.cos(2 * theta)))
                    / (
                        (a**2 + r * (-2 * M + r))
                        * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2
                    ),
                    (-8 * a * M * r * (sme.cos(theta) / sme.sin(theta)))
                    / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2,
                    0,
                ],
                [
                    (-2 * a * M * (a**2 - 2 * r**2 + a**2 * sme.cos(2 * theta)))
                    / (
                        (a**2 + r * (-2 * M + r))
                        * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2
                    ),
                    0,
                    0,
                    (
                        a**4 * M
                        + 3 * a**4 * r
                        - 12 * a**2 * M * r**2
                        + 8 * a**2 * r**3
                        - 16 * M * r**4
                        + 8 * r**5
                        + 4 * a**2 * r * (a**2 + r * (-M + 2 * r)) * sme.cos(2 * theta)
                        - a**4 * (M - r) * sme.cos(4 * theta)
                    )
                    / (
                        2
                        * (a**2 + r * (-2 * M + r))
                        * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2
                    ),
                ],
                [
                    (-8 * a * M * r * (sme.cos(theta) / sme.sin(theta)))
                    / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2,
                    0,
                    0,
                    (
                        (
                            3 * a**4
                            + 8 * a**2 * M * r
                            + 8 * a**2 * r**2
                            + 8 * r**4
                            + 4 * a**2 * (a**2 + 2 * r * (-M + r)) * sme.cos(2 * theta)
                            + a**4 * sme.cos(4 * theta)
                        )
                        * (sme.cos(theta) / sme.sin(theta))
                    )
                    / (2 * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2),
                ],
                [
                    0,
                    (
                        a**4 * M
                        + 3 * a**4 * r
                        - 12 * a**2 * M * r**2
                        + 8 * a**2 * r**3
                        - 16 * M * r**4
                        + 8 * r**5
                        + 4 * a**2 * r * (a**2 + r * (-M + 2 * r)) * sme.cos(2 * theta)
                        - a**4 * (M - r) * sme.cos(4 * theta)
                    )
                    / (
                        2
                        * (a**2 + r * (-2 * M + r))
                        * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2
                    ),
                    (
                        (
                            3 * a**4
                            + 8 * a**2 * M * r
                            + 8 * a**2 * r**2
                            + 8 * r**4
                            + 4 * a**2 * (a**2 + 2 * r * (-M + r)) * sme.cos(2 * theta)
                            + a**4 * sme.cos(4 * theta)
                        )
                        * (sme.cos(theta) / sme.sin(theta))
                    )
                    / (2 * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 2),
                    0,
                ],
            ],
        ]
    )

In [165]:
def kerr_riemann_tensor(r, theta, a, M=1.0, config="ulll"):
    """
    Define variables

    Components of the Riemann tensor for Kerr Metric
    From Reference Paper, Appendix
    The Configuration is ulll
    """

    rijkl = np.zeros((4, 4, 4, 4), dtype=type(r))

    # Definitions
    Delta = a**2 - 2.0 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    rijkl[0, 0, 0, 3] = (
        2.0
        * M**2
        * sme.sin(theta) ** 2
        * a
        * r**2
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / Sigma**4
    )

    rijkl[0, 0, 1, 2] = (
        -2.0
        * M**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * r
        * sme.cos(theta)
        * a**2
        * sme.sin(theta)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 0, 1] = (
        M
        * r
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 0, 2] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2 * M * r)
        * a**2
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 1, 3] = (
        3.0
        * M
        * sme.sin(theta) ** 2
        * a
        * r
        * (r**2 + a**2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 2, 3] = (
        -M
        * sme.cos(theta)
        * sme.sin(theta)
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (2.0 * (r**2 + a**2) ** 2 + a**2 * sme.sin(theta) ** 2 * Delta)
        / (Sigma**3 * Delta)
    )

    # Antisymmetric under exchange of last two indices.
    rijkl[0, 0, 3, 0] = -rijkl[0, 0, 0, 3]
    rijkl[0, 0, 2, 1] = -rijkl[0, 0, 1, 2]
    rijkl[0, 1, 1, 0] = -rijkl[0, 1, 0, 1]
    rijkl[0, 1, 2, 0] = -rijkl[0, 1, 0, 2]
    rijkl[0, 1, 3, 1] = -rijkl[0, 1, 1, 3]
    rijkl[0, 1, 3, 2] = -rijkl[0, 1, 2, 3]

    #! -------------------------------------------------------------------------------------#

    rijkl[0, 2, 0, 1] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 4 * M * r)
        * a**2
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )
    rijkl[0, 2, 0, 2] = (
        -M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2 * a**2 * sme.sin(theta) ** 2)
        / Sigma**3
    )
    rijkl[0, 2, 1, 3] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * ((a**2 + r**2) ** 2 + 2.0 * a**2 * sme.sin(theta) ** 2 * Delta)
        * a
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )
    rijkl[0, 2, 2, 3] = (
        -3.0
        * M
        * sme.sin(theta) ** 2
        * r
        * a
        * (a**2 + r**2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / Sigma**3
    )
    rijkl[0, 3, 0, 3] = (
        -sme.sin(theta) ** 2
        * M
        * r
        * (r**2 + 3 * a**2 * sme.sin(theta) ** 2 - 3 * a**2)
        * ((a**2 + r**2) ** 2 - a**2 * sme.sin(theta) ** 2 * Delta)
        / Sigma**4
    )
    rijkl[0, 3, 1, 2] = (
        (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * M
        * ((a**2 + r**2) ** 2 - a**2 * sme.sin(theta) ** 2 * Delta)
        * a
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )

    # Symmetries
    rijkl[0, 2, 1, 0] = -rijkl[0, 2, 0, 1]
    rijkl[0, 2, 2, 0] = -rijkl[0, 2, 0, 2]
    rijkl[0, 2, 3, 1] = -rijkl[0, 2, 1, 3]
    rijkl[0, 2, 3, 2] = -rijkl[0, 2, 2, 3]
    rijkl[0, 3, 3, 0] = -rijkl[0, 3, 0, 3]
    rijkl[0, 3, 2, 1] = -rijkl[0, 3, 1, 2]

    #! ------------------------------------------------------------------------------#

    rijkl[1, 0, 0, 1] = (
        r
        * M
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (a**2 * sme.sin(theta) ** 2 + 2.0 * Delta)
        / Sigma**4
    )
    rijkl[1, 0, 0, 2] = (
        -3.0
        * M
        * Delta
        * a**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta)
        * sme.cos(theta)
        / Sigma**4
    )
    rijkl[1, 0, 1, 3] = (
        r
        * M
        * a
        * (3.0 * (r**2 + a**2) - 4.0 * M * r)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[1, 0, 2, 3] = (
        -a
        * M
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * Delta
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[1, 2, 0, 3] = (
        -sme.cos(theta)
        * M
        * sme.sin(theta)
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * Delta
        / Sigma**3
    )
    rijkl[1, 2, 1, 2] = -M * r * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / Sigma**2
    rijkl[1, 3, 0, 1] = (
        -sme.sin(theta) ** 2
        * M
        * r
        * a
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 4.0 * M * r)
        / Sigma**4
    )
    rijkl[1, 3, 0, 2] = (
        a
        * M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2.0 * a**2 * sme.sin(theta) ** 2)
        * Delta
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[1, 3, 1, 3] = (
        -sme.sin(theta) ** 2
        * M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * ((r**2 + a**2) ** 2 + 2.0 * a**2 * Delta * sme.sin(theta) ** 2)
        / Sigma**4
    )
    rijkl[1, 3, 2, 3] = (
        3.0
        * M
        * a**2
        * (r**2 + a**2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * Delta
        * sme.cos(theta)
        * sme.sin(theta) ** 3
        / Sigma**4
    )

    # Symmetries
    rijkl[1, 0, 1, 0] = -rijkl[1, 0, 0, 1]
    rijkl[1, 0, 2, 0] = -rijkl[1, 0, 0, 2]
    rijkl[1, 0, 3, 1] = -rijkl[1, 0, 1, 3]
    rijkl[1, 0, 3, 2] = -rijkl[1, 0, 2, 3]
    rijkl[1, 2, 3, 0] = -rijkl[1, 2, 0, 3]
    rijkl[1, 2, 2, 1] = -rijkl[1, 2, 1, 2]
    rijkl[1, 3, 1, 0] = -rijkl[1, 3, 0, 1]
    rijkl[1, 3, 2, 0] = -rijkl[1, 3, 0, 2]
    rijkl[1, 3, 3, 1] = -rijkl[1, 3, 1, 3]
    rijkl[1, 3, 3, 2] = -rijkl[1, 3, 2, 3]

    #! -------------------------------------------------------------------------------------#

    rijkl[2, 0, 0, 1] = (
        -3.0
        * M
        * a**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta)
        * sme.cos(theta)
        / Sigma**4
    )
    rijkl[2, 0, 0, 2] = (
        -M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (2.0 * a**2 * sme.sin(theta) ** 2 + Delta)
        / Sigma**4
    )
    rijkl[2, 0, 1, 3] = (
        -a
        * M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2.0 * a**2 * sme.sin(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[2, 0, 2, 3] = (
        -M
        * r
        * a
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2.0 * M * r)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[2, 1, 0, 3] = (
        M
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**3
    )
    rijkl[2, 1, 1, 2] = (
        M * r * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / (Delta * Sigma**2)
    )
    rijkl[2, 3, 0, 1] = (
        M
        * a
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[2, 3, 0, 2] = (
        M
        * r
        * a
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2.0 * M * r)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[2, 3, 1, 3] = (
        3.0
        * M
        * a**2
        * (r**2 + a**2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta) ** 3
        / Sigma**4
    )
    rijkl[2, 3, 2, 3] = (
        M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (2.0 * (r**2 + a**2) ** 2 + a**2 * Delta * sme.sin(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**4
    )

    # Symmetries
    rijkl[2, 0, 1, 0] = -rijkl[2, 0, 0, 1]
    rijkl[2, 0, 2, 0] = -rijkl[2, 0, 0, 2]
    rijkl[2, 0, 3, 1] = -rijkl[2, 0, 1, 3]
    rijkl[2, 0, 3, 2] = -rijkl[2, 0, 2, 3]
    rijkl[2, 1, 3, 0] = -rijkl[2, 1, 0, 3]
    rijkl[2, 1, 2, 1] = -rijkl[2, 1, 1, 2]
    rijkl[2, 3, 1, 0] = -rijkl[2, 3, 0, 1]
    rijkl[2, 3, 2, 0] = -rijkl[2, 3, 0, 2]
    # rijkl[2, 3, 2, 1] = -rijkl[2, 3, 1, 2]
    rijkl[2, 3, 3, 1] = -rijkl[2, 3, 1, 3]
    rijkl[2, 3, 3, 2] = -rijkl[2, 3, 2, 3]

    #! --------------------------------------------------------------------------#

    rijkl[3, 0, 1, 2] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (a**2 * sme.sin(theta) ** 2 - Delta)
        * a
        * (sme.cos(theta) / sme.sin(theta))
        / (Delta * Sigma**3)
    )
    rijkl[3, 1, 0, 1] = (
        3.0 * M * r * a * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / (Delta * Sigma**3)
    )
    rijkl[3, 1, 0, 2] = (
        -M
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (2.0 * a**2 * sme.sin(theta) ** 2 + Delta)
        * (sme.cos(theta) / sme.sin(theta))
        / (Delta * Sigma**3)
    )
    rijkl[3, 1, 1, 3] = (
        M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2.0 * a**2 * sme.sin(theta) ** 2)
        / (Delta * Sigma**3)
    )
    rijkl[3, 1, 2, 3] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2.0 * M * r)
        * a**2
        * sme.sin(theta)
        * sme.cos(theta)
        / (Delta * Sigma**3)
    )
    rijkl[3, 2, 0, 1] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (a**2 * sme.sin(theta) ** 2 + 2.0 * Delta)
        * a
        * (sme.cos(theta) / sme.sin(theta))
        / (Delta * Sigma**3)
    )
    rijkl[3, 2, 0, 2] = (
        -M * 3.0 * r * a * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / Sigma**3
    )
    rijkl[3, 2, 1, 3] = (
        -M
        * a**2
        * (3 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 4.0 * M * r)
        * sme.sin(theta)
        * sme.cos(theta)
        / (Delta * Sigma**3)
    )
    rijkl[3, 2, 2, 3] = (
        -M
        * r
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / Sigma**3
    )
    rijkl[3, 3, 0, 3] = (
        -2.0
        * M**2
        * a
        * r**2
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[3, 3, 1, 2] = (
        2.0
        * M**2
        * r
        * a**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / (Delta * Sigma**3)
    )

    # Symmetries
    rijkl[3, 0, 2, 1] = -rijkl[3, 0, 1, 2]
    rijkl[3, 1, 1, 0] = -rijkl[3, 1, 0, 1]
    rijkl[3, 1, 2, 0] = -rijkl[3, 1, 0, 2]
    rijkl[3, 1, 3, 1] = -rijkl[3, 1, 1, 3]
    rijkl[3, 1, 3, 2] = -rijkl[3, 1, 2, 3]
    rijkl[3, 2, 1, 0] = -rijkl[3, 2, 0, 1]
    rijkl[3, 2, 2, 0] = -rijkl[3, 2, 0, 2]
    rijkl[3, 2, 3, 1] = -rijkl[3, 2, 1, 3]
    rijkl[3, 2, 3, 2] = -rijkl[3, 2, 2, 3]
    rijkl[3, 3, 3, 0] = -rijkl[3, 3, 0, 3]
    rijkl[3, 3, 2, 1] = -rijkl[3, 3, 1, 2]

    #!---------------------------------------------------------------------#

    part1 = a**2 + 2 * r * (-2 * M + r) + a**2 * sme.cos(2 * theta)
    part2 = 3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta)
    part3 = (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4
    # Calculating Rtensor elements

    # rijkl[3, 0, 0, 3] =  (part1 + part2 + part3 + part4) / (Sigma**5)
    rijkl[3, 0, 0, 3] = 4 * M * r * part1 * part2 / part3
    rijkl[3, 0, 3, 0] = -rijkl[3, 0, 0, 3]

    # return rijkl
    if config == "ulll":
        return rijkl + np.zeros((4, 4, 4, 4))

    elif config == "llll":
        gk = cov_metric_tensor(r=r, theta=theta, a=a, M=M)
        return np.einsum("ij,jklm->iklm", gk, rijkl) + np.zeros((4, 4, 4, 4))

    # elif config=="lluu":
    #     #!Config: lluu
    #     gkinv = cot_metric_tensor(r=r, theta=theta, a=a, M=M)
    #     gk = cov_metric_tensor(r=r, theta=theta, a=a, M=M)
    #     rllll = np.einsum("ij,jklm->iklm", gk, rijkl)

    #     return np.einsum("kjlm,la,mb->kjab", rllll, gkinv, gkinv)

    else:
        return "Invalid String"

In [166]:
# @nb.njit
def Bkerr_riemann_tensor(r, theta, a, M):
    # rijkl= np.zeros((4,4,4,4))
    rijkl = np.array(
        [
            [
                [
                    [
                        0,
                        0,
                        0,
                        (
                            -16
                            * a
                            * M**2
                            * r**2
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        0,
                        0,
                        (
                            4
                            * a**2
                            * M**2
                            * r
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                    ],
                    [
                        0,
                        (
                            -4
                            * a**2
                            * M**2
                            * r
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                        0,
                    ],
                    [
                        (
                            16
                            * a
                            * M**2
                            * r**2
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        0,
                    ],
                ],
                [
                    [
                        0,
                        (
                            M
                            * r
                            * (
                                -27 * a**4
                                - 4 * a**2 * r**2
                                + 16 * r**4
                                - 4 * a**2 * (6 * a**2 + 7 * r**2) * sme.cos(2 * theta)
                                + 3 * a**4 * sme.cos(4 * theta)
                            )
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        (
                            2
                            * a**2
                            * M
                            * (3 * a**2 + r * (-2 * M + 3 * r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                    ],
                    [
                        -(
                            (
                                M
                                * r
                                * (
                                    -27 * a**4
                                    - 4 * a**2 * r**2
                                    + 16 * r**4
                                    - 4
                                    * a**2
                                    * (6 * a**2 + 7 * r**2)
                                    * sme.cos(2 * theta)
                                    + 3 * a**4 * sme.cos(4 * theta)
                                )
                            )
                            / (
                                (a**2 + r * (-2 * M + r))
                                * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                            )
                        ),
                        0,
                        0,
                        (
                            -12
                            * a
                            * M
                            * r
                            * (a**2 + r**2)
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                    ],
                    [
                        (
                            -2
                            * a**2
                            * M
                            * (3 * a**2 + r * (-2 * M + 3 * r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                        0,
                        -(
                            (
                                a
                                * M
                                * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                                * (
                                    -5 * a**4
                                    + a**2 * (2 * M - 9 * r) * r
                                    - 4 * r**4
                                    + a**2
                                    * (a**2 + r * (-2 * M + r))
                                    * sme.cos(2 * theta)
                                )
                                * sme.sin(2 * theta)
                            )
                            / (
                                (a**2 + r * (-2 * M + r))
                                * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                            )
                        ),
                    ],
                    [
                        0,
                        (
                            12
                            * a
                            * M
                            * r
                            * (a**2 + r**2)
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        (
                            a
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * (
                                -5 * a**4
                                + a**2 * (2 * M - 9 * r) * r
                                - 4 * r**4
                                + a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                    ],
                ],
                [
                    [
                        0,
                        (
                            2
                            * a**2
                            * M
                            * (3 * a**2 + r * (-4 * M + 3 * r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        (
                            -2
                            * M
                            * r
                            * (
                                -9 * a**4
                                + 2 * a**2 * r**2
                                + 4 * r**4
                                - 2 * a**2 * (3 * a**2 + 5 * r**2) * sme.cos(2 * theta)
                                + 3 * a**4 * sme.cos(4 * theta)
                            )
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                        0,
                    ],
                    [
                        (
                            -2
                            * a**2
                            * M
                            * (3 * a**2 + r * (-4 * M + 3 * r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                        0,
                        (
                            -2
                            * a
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * (
                                -2 * a**4
                                + a**2 * (2 * M - 3 * r) * r
                                - r**4
                                + a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                    ],
                    [
                        (
                            2
                            * M
                            * r
                            * (
                                -9 * a**4
                                + 2 * a**2 * r**2
                                + 4 * r**4
                                - 2 * a**2 * (3 * a**2 + 5 * r**2) * sme.cos(2 * theta)
                                + 3 * a**4 * sme.cos(4 * theta)
                            )
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                        0,
                        0,
                        (
                            12
                            * a
                            * M
                            * r
                            * (a**2 + r**2)
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                    ],
                    [
                        0,
                        (
                            2
                            * a
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * (
                                -2 * a**4
                                + a**2 * (2 * M - 3 * r) * r
                                - r**4
                                + a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        (
                            -12
                            * a
                            * M
                            * r
                            * (a**2 + r**2)
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                        0,
                    ],
                ],
                [
                    [
                        0,
                        0,
                        0,
                        (
                            4
                            * M
                            * r
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * (
                                a**4
                                + 2 * r**4
                                + a**2 * r * (2 * M + 3 * r)
                                + a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                            )
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        0,
                        0,
                        -(
                            (
                                a
                                * M
                                * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                                * (
                                    a**4
                                    + 2 * r**4
                                    + a**2 * r * (2 * M + 3 * r)
                                    + a**2
                                    * (a**2 + r * (-2 * M + r))
                                    * sme.cos(2 * theta)
                                )
                                * sme.sin(2 * theta)
                            )
                            / (
                                (a**2 + r * (-2 * M + r))
                                * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                            )
                        ),
                        0,
                    ],
                    [
                        0,
                        (
                            a
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * (
                                a**4
                                + 2 * r**4
                                + a**2 * r * (2 * M + 3 * r)
                                + a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                        0,
                    ],
                    [
                        (
                            -4
                            * M
                            * r
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * (
                                a**4
                                + 2 * r**4
                                + a**2 * r * (2 * M + 3 * r)
                                + a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                            )
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        0,
                    ],
                ],
            ],
            [
                [
                    [
                        0,
                        (
                            4
                            * M
                            * r
                            * (
                                -5 * a**2
                                + 8 * M * r
                                - 4 * r**2
                                + a**2 * sme.cos(2 * theta)
                            )
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        (
                            12
                            * a**2
                            * M
                            * (a**2 + r * (-2 * M + r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                    ],
                    [
                        (
                            4
                            * M
                            * r
                            * (-3 * a**2 + 2 * r**2 - 3 * a**2 * sme.cos(2 * theta))
                            * (
                                -5 * a**2
                                + 8 * M * r
                                - 4 * r**2
                                + a**2 * sme.cos(2 * theta)
                            )
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        (
                            -8
                            * a
                            * M
                            * r
                            * (3 * a**2 + r * (-4 * M + 3 * r))
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        (
                            -12
                            * a**2
                            * M
                            * (a**2 + r * (-2 * M + r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        (
                            a
                            * M
                            * (a**2 + r * (-2 * M + r))
                            * (
                                9 * a**4
                                - 52 * a**2 * r**2
                                - 48 * r**4
                                + 4 * a**2 * (2 * a**2 + 5 * r**2) * sme.cos(2 * theta)
                                - a**4 * sme.cos(4 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        0,
                        (
                            8
                            * a
                            * M
                            * r
                            * (3 * a**2 + r * (-4 * M + 3 * r))
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        (
                            a
                            * M
                            * (a**2 + r * (-2 * M + r))
                            * (
                                -9 * a**4
                                + 52 * a**2 * r**2
                                + 48 * r**4
                                - 4 * a**2 * (2 * a**2 + 5 * r**2) * sme.cos(2 * theta)
                                + a**4 * sme.cos(4 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                    ],
                ],
                [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
                [
                    [
                        0,
                        0,
                        0,
                        (
                            2
                            * a
                            * M
                            * (a**2 + r * (-2 * M + r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                    ],
                    [
                        0,
                        0,
                        (M * r * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta)))
                        / (2 * (r**2 + a**2 * sme.cos(theta) ** 2) ** 2),
                        0,
                    ],
                    [
                        0,
                        (M * r * (-3 * a**2 + 2 * r**2 - 3 * a**2 * sme.cos(2 * theta)))
                        / (2 * (r**2 + a**2 * sme.cos(theta) ** 2) ** 2),
                        0,
                        0,
                    ],
                    [
                        (
                            -2
                            * a
                            * M
                            * (a**2 + r * (-2 * M + r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                        0,
                        0,
                        0,
                    ],
                ],
                [
                    [
                        0,
                        (
                            8
                            * a
                            * M
                            * r
                            * (3 * a**2 + r * (-4 * M + 3 * r))
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        (
                            2
                            * a
                            * M
                            * (a**2 + r * (-2 * M + r))
                            * (
                                -3 * a**4
                                + 22 * a**2 * r**2
                                + 12 * r**4
                                - 2 * a**2 * (a**2 + 7 * r**2) * sme.cos(2 * theta)
                                + a**4 * sme.cos(4 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                    ],
                    [
                        (
                            -8
                            * a
                            * M
                            * r
                            * (3 * a**2 + r * (-4 * M + 3 * r))
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        (
                            8
                            * M
                            * r
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * (
                                2 * a**4
                                + r**4
                                + a**2 * r * (-2 * M + 3 * r)
                                - a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                            )
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        (
                            -2
                            * a
                            * M
                            * (a**2 + r * (-2 * M + r))
                            * (
                                -3 * a**4
                                + 22 * a**2 * r**2
                                + 12 * r**4
                                - 2 * a**2 * (a**2 + 7 * r**2) * sme.cos(2 * theta)
                                + a**4 * sme.cos(4 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        (
                            -24
                            * a**2
                            * M
                            * (a**2 + r**2)
                            * (a**2 + r * (-2 * M + r))
                            * sme.cos(theta)
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 3
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        0,
                        (
                            -8
                            * M
                            * r
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * (
                                2 * a**4
                                + r**4
                                + a**2 * r * (-2 * M + 3 * r)
                                - a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                            )
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        (
                            24
                            * a**2
                            * M
                            * (a**2 + r**2)
                            * (a**2 + r * (-2 * M + r))
                            * sme.cos(theta)
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 3
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                    ],
                ],
            ],
            [
                [
                    [
                        0,
                        (
                            12
                            * a**2
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        (
                            8
                            * M
                            * r
                            * (-3 * a**2 + 2 * r**2 - 3 * a**2 * sme.cos(2 * theta))
                            * (-2 * a**2 + (2 * M - r) * r + a**2 * sme.cos(2 * theta))
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                    ],
                    [
                        (
                            -12
                            * a**2
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        (
                            -2
                            * a
                            * M
                            * (
                                -3 * a**4
                                + 22 * a**2 * r**2
                                + 12 * r**4
                                - 2 * a**2 * (a**2 + 7 * r**2) * sme.cos(2 * theta)
                                + a**4 * sme.cos(4 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        (
                            8
                            * M
                            * r
                            * (-3 * a**2 + 2 * r**2 - 3 * a**2 * sme.cos(2 * theta))
                            * (2 * a**2 + r * (-2 * M + r) - a**2 * sme.cos(2 * theta))
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        (
                            8
                            * a
                            * M
                            * r
                            * (3 * a**2 + r * (-2 * M + 3 * r))
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        0,
                        (
                            2
                            * a
                            * M
                            * (
                                -3 * a**4
                                + 22 * a**2 * r**2
                                + 12 * r**4
                                - 2 * a**2 * (a**2 + 7 * r**2) * sme.cos(2 * theta)
                                + a**4 * sme.cos(4 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        (
                            -8
                            * a
                            * M
                            * r
                            * (3 * a**2 + r * (-2 * M + 3 * r))
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                    ],
                ],
                [
                    [
                        0,
                        0,
                        0,
                        (
                            -2
                            * a
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                    ],
                    [
                        0,
                        0,
                        (M * r * (-3 * a**2 + 2 * r**2 - 3 * a**2 * sme.cos(2 * theta)))
                        / (
                            2
                            * (a**2 + r * (-2 * M + r))
                            * (r**2 + a**2 * sme.cos(theta) ** 2) ** 2
                        ),
                        0,
                    ],
                    [
                        0,
                        (M * r * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta)))
                        / (
                            2
                            * (a**2 + r * (-2 * M + r))
                            * (r**2 + a**2 * sme.cos(theta) ** 2) ** 2
                        ),
                        0,
                        0,
                    ],
                    [
                        (
                            2
                            * a
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                        0,
                        0,
                        0,
                    ],
                ],
                [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
                [
                    [
                        0,
                        (
                            a
                            * M
                            * (
                                -9 * a**4
                                + 52 * a**2 * r**2
                                + 48 * r**4
                                - 4 * a**2 * (2 * a**2 + 5 * r**2) * sme.cos(2 * theta)
                                + a**4 * sme.cos(4 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        (
                            -8
                            * a
                            * M
                            * r
                            * (3 * a**2 + r * (-2 * M + 3 * r))
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                    ],
                    [
                        (
                            a
                            * M
                            * (
                                9 * a**4
                                - 52 * a**2 * r**2
                                - 48 * r**4
                                + 4 * a**2 * (2 * a**2 + 5 * r**2) * sme.cos(2 * theta)
                                - a**4 * sme.cos(4 * theta)
                            )
                            * sme.sin(2 * theta)
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        (
                            -24
                            * a**2
                            * M
                            * (a**2 + r**2)
                            * sme.cos(theta)
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 3
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        (
                            8
                            * a
                            * M
                            * r
                            * (3 * a**2 + r * (-2 * M + 3 * r))
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        (
                            -4
                            * M
                            * r
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * (
                                5 * a**4
                                + 4 * r**4
                                + a**2 * r * (-2 * M + 9 * r)
                                - a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                            )
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        0,
                        (
                            24
                            * a**2
                            * M
                            * (a**2 + r**2)
                            * sme.cos(theta)
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 3
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        (
                            4
                            * M
                            * r
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * (
                                5 * a**4
                                + 4 * r**4
                                + a**2 * r * (-2 * M + 9 * r)
                                - a**2 * (a**2 + r * (-2 * M + r)) * sme.cos(2 * theta)
                            )
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                    ],
                ],
            ],
            [
                [
                    [
                        0,
                        0,
                        0,
                        (
                            4
                            * M
                            * r
                            * (a**2 + 2 * r * (-2 * M + r) + a**2 * sme.cos(2 * theta))
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        0,
                        0,
                        (
                            -2
                            * a
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * (a**2 + 2 * r * (-2 * M + r) + a**2 * sme.cos(2 * theta))
                            * (sme.cos(theta) / sme.sin(theta))
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                    ],
                    [
                        0,
                        (
                            2
                            * a
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * (a**2 + 2 * r * (-2 * M + r) + a**2 * sme.cos(2 * theta))
                            * (sme.cos(theta) / sme.sin(theta))
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                        0,
                    ],
                    [
                        (
                            -4
                            * M
                            * r
                            * (a**2 + 2 * r * (-2 * M + r) + a**2 * sme.cos(2 * theta))
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        0,
                    ],
                ],
                [
                    [
                        0,
                        (
                            -12
                            * a
                            * M
                            * r
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        (
                            -4
                            * a
                            * M
                            * (-2 * a**2 + (2 * M - r) * r + a**2 * sme.cos(2 * theta))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * (sme.cos(theta) / sme.sin(theta))
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                    ],
                    [
                        (
                            12
                            * a
                            * M
                            * r
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                        0,
                        (
                            2
                            * M
                            * r
                            * (
                                -9 * a**4
                                + 2 * a**2 * r**2
                                + 4 * r**4
                                - 2 * a**2 * (3 * a**2 + 5 * r**2) * sme.cos(2 * theta)
                                + 3 * a**4 * sme.cos(4 * theta)
                            )
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                    ],
                    [
                        (
                            4
                            * a
                            * M
                            * (-2 * a**2 + (2 * M - r) * r + a**2 * sme.cos(2 * theta))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * (sme.cos(theta) / sme.sin(theta))
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                        0,
                        (
                            2
                            * a**2
                            * M
                            * (3 * a**2 + r * (-2 * M + 3 * r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                    ],
                    [
                        0,
                        (
                            -2
                            * M
                            * r
                            * (
                                -9 * a**4
                                + 2 * a**2 * r**2
                                + 4 * r**4
                                - 2 * a**2 * (3 * a**2 + 5 * r**2) * sme.cos(2 * theta)
                                + 3 * a**4 * sme.cos(4 * theta)
                            )
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        (
                            -2
                            * a**2
                            * M
                            * (3 * a**2 + r * (-2 * M + 3 * r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                    ],
                ],
                [
                    [
                        0,
                        (
                            -2
                            * a
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * (
                                -5 * a**2
                                + 8 * M * r
                                - 4 * r**2
                                + a**2 * sme.cos(2 * theta)
                            )
                            * (sme.cos(theta) / sme.sin(theta))
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        (
                            12
                            * a
                            * M
                            * r
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                        0,
                    ],
                    [
                        (
                            2
                            * a
                            * M
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * (
                                -5 * a**2
                                + 8 * M * r
                                - 4 * r**2
                                + a**2 * sme.cos(2 * theta)
                            )
                            * (sme.cos(theta) / sme.sin(theta))
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                        0,
                        (
                            2
                            * a**2
                            * M
                            * (3 * a**2 + r * (-4 * M + 3 * r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                    ],
                    [
                        (
                            -12
                            * a
                            * M
                            * r
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                        0,
                        0,
                        -(
                            (
                                M
                                * r
                                * (
                                    -27 * a**4
                                    - 4 * a**2 * r**2
                                    + 16 * r**4
                                    - 4
                                    * a**2
                                    * (6 * a**2 + 7 * r**2)
                                    * sme.cos(2 * theta)
                                    + 3 * a**4 * sme.cos(4 * theta)
                                )
                            )
                            / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                    ],
                    [
                        0,
                        (
                            -2
                            * a**2
                            * M
                            * (3 * a**2 + r * (-4 * M + 3 * r))
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        (
                            M
                            * r
                            * (
                                -27 * a**4
                                - 4 * a**2 * r**2
                                + 16 * r**4
                                - 4 * a**2 * (6 * a**2 + 7 * r**2) * sme.cos(2 * theta)
                                + 3 * a**4 * sme.cos(4 * theta)
                            )
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3,
                        0,
                    ],
                ],
                [
                    [
                        0,
                        0,
                        0,
                        (
                            16
                            * a
                            * M**2
                            * r**2
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                    ],
                    [
                        0,
                        0,
                        (
                            -4
                            * a**2
                            * M**2
                            * r
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                    ],
                    [
                        0,
                        (
                            4
                            * a**2
                            * M**2
                            * r
                            * (a**2 - 6 * r**2 + a**2 * sme.cos(2 * theta))
                            * sme.sin(2 * theta)
                        )
                        / (
                            (a**2 + r * (-2 * M + r))
                            * (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 3
                        ),
                        0,
                        0,
                    ],
                    [
                        (
                            -16
                            * a
                            * M**2
                            * r**2
                            * (3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta))
                            * sme.sin(theta) ** 2
                        )
                        / (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4,
                        0,
                        0,
                        0,
                    ],
                ],
            ],
        ]
    )

    return rijkl

In [167]:
def spintensor(levi, pvector, svector, m):

    # p_vectorl = np.einsum("ij,j->i", gk, pvector)  #! Lower the pvector
    # s_vectorl = np.einsum("ij,j->i", gk, svector)  #! Lower the svector

    #! Using above definition
    return -np.einsum("uvab,a,b->uv", levi, svector, pvector) / m

In [168]:
def metric_D(r, theta, a, M=1.0):
    dgkt = np.zeros((4, 4), dtype=type(r))
    dgkp = np.zeros((4, 4), dtype=type(r))

    denom = (r**2 + a**2 * sme.cos(theta) ** 2) ** 2

    dgkt[1, 0] = M * (a**2 - 2 * r**2 + a**2 * sme.cos(2 * theta)) / denom
    dgkt[1, 3] = (
        -(2 * a * M * (-(r**2) + a**2 * sme.cos(theta) ** 2) * sme.sin(theta) ** 2)
        / denom
    )
    dgkt[2, 0] = 2 * a**2 * M * r * sme.sin(2 * theta) / denom
    dgkt[2, 3] = -2 * a * M * r * (r**2 + a**2) * sme.sin(2 * theta) / denom

    dgkp[1, 0] = (
        -2 * a * M * (-(r**2) + a**2 * sme.cos(theta) ** 2) * sme.sin(theta) ** 2 / denom
    )

    dgkp[1, 3] = (
        2
        * sme.sin(theta) ** 2
        * (
            sme.cos(theta) ** 2
            * (2 * a**2 * r * (a**2 + r**2) + a**4 * (M - r) * sme.sin(theta) ** 2)
            + r * (-(a**4) + r**4 + a**2 * (a**2 - M * r) * sme.sin(theta) ** 2)
        )
        / denom
    )

    dgkp[2, 0] = -2 * a * M * r * (a**2 + r**2) * sme.sin(2 * theta) / denom
    dgkp[2, 3] = (
        (
            3 * a**6
            + 10 * a**4 * M * r
            + 11 * a**4 * r**2
            + 16 * a**2 * M * r**3
            + 16 * a**2 * r**4
            + 8 * r**6
            + 4
            * a**2
            * (a**2 + 2 * r**2)
            * (a**2 + r * (-2 * M + r))
            * sme.cos(2 * theta)
            + a**4 * (a**2 - 2 * M * r + r**2) * sme.cos(4 * theta)
        )
        * sme.sin(2 * theta)
        / (8 * denom)
    )

    return dgkt.T + np.zeros((4, 4)), dgkp.T + np.zeros((4, 4))

In [169]:
# @snoop
def energy(pvector, Stensor, dgkt):
    #! Changed Here
    # pvecl = np.einsum("ij,j->i", gk, pvector)
    term2 = np.einsum("uv,uv->", dgkt, Stensor) / 2.0
    return -pvector[0] + term2

In [170]:
def angular_momentum(pvector, Stensor, dgkp):
    #! Changed Here
    # pvecl = np.einsum("ij,j->i", gk, pvector)
    term2 = np.einsum("uv,uv->", dgkp, Stensor) / 2.0
    return pvector[3] - term2

In [171]:
def calculate_four_momentum(pvector, uvector, svector, cs, dRlluu):
    correction = -np.einsum("uvab,v,a,b->u", dRlluu, uvector, pvector, svector)
    # correction = -dRlluu.dot(svector).dot(pvector).dot(uvector)
    dp = correction + oe.contract("auv,a,u->v", cs, pvector, uvector, backend="object")
    # dp = correction + np.einsum("uv,u->v", np.einsum("auv,a->uv", cs, pvector), uvector)
    # dp = correction + cs.transpose(2, 0, 1).dot(uvector).dot(pvector)

    return dp

In [172]:
# @snoop
def calculate_four_spin(pvector, uvector, svector, cs, dRuluu):

    # correction= -np.einsum("u,abgd,a,b,g,d->u",pvector,dRuluu,svector,uvector,pvector,svector)
    correction = -pvector * dRuluu.dot(svector).dot(pvector).dot(uvector).dot(svector)

    ds = correction + oe.contract("auv,a,u->v", cs, svector, uvector,backend='object')
    # ds = correction + cs.transpose(2, 0, 1).dot(uvector).dot(svector)

    return ds

In [173]:
# @snoop
def calculate_four_velocity(gk, gkinv, pvector, svector, ddRuuuu,return_N=False):

    # correction= -np.einsum("uabg,a,b,g->u",ddRuuuu,svector,pvector,svector)
    correction = -ddRuuuu.dot(svector).dot(pvector).dot(svector)

    # pvecu= np.einsum("ij,j->i",gkinv,pvector)
    pvecu = gkinv.dot(pvector)

    N = sme.sqrt(
        1.0 / (1.0 - gk.dot(correction).dot(correction) - 2.0 * pvector.dot(correction))
    )

    dx = N * (pvecu + correction)

    if return_N:
        return dx,N
    return dx

In [174]:
# @snoop
def MPD(t, y, p):
    t, r, theta, phi, pt, pr, ptheta, pphi, st, sr, stheta, sphi = y

    a, m, M = p

    # xvector=y[:4]
    pvector = y[4:8]  #! p_u
    svector = y[8:]  #! s_u

    gk = Bcov_metric_tensor(r, theta, a, M)  #! g_{ij}
    gkinv = Bcot_metric_tensor(r, theta, a, M)
    cs = Bkerr_christoffel(r, theta, a, M)  #! G^i_{jk}

    Riemann = Bkerr_riemann_tensor(r, theta, a, M)  #! R^a_{bcd}

    # Riemann_cov = np.einsum("ul,lvps->uvps", gk, Riemann)  #! R_{abcd}
    # Riemann_uull = np.einsum("ap,upgd->uagd", gkinv, Riemann)  #! R^{ua}_{gd}

    levi = levi_civita_tensor(gk)  #! e^{abcd}
    # levi_mixed = np.einsum("psxy,xu,yv->psuv", levi, gk, gk)  #! e^{ps}_{uv}
    levi_mixed = np.einsum("px,sy,xyuv->psuv", gk, gk, levi)

    dRuluu = 0.5 * np.einsum("abps,psuv->abuv", Riemann, levi)
    dRlluu = np.einsum("ia,abuv->ibuv", gk, dRuluu)

    ddRuuuu = 0.5 * np.einsum("bi,aips,psuv->abuv", gkinv, dRuluu, levi_mixed)

    # stensor = spintensor(levi, pvector=pvector, svector=svector, gk=gk, m=m)  #! S^{uv}

    uvector = calculate_four_velocity(
        gk=gk, gkinv=gkinv, pvector=pvector, svector=svector, ddRuuuu=ddRuuuu
    )  #!dx_mu

    dp = calculate_four_momentum(
        pvector=pvector, uvector=uvector, svector=svector, cs=cs, dRlluu=dRlluu
    )
    #!dp_{mu}

    ds = calculate_four_spin(
        pvector=pvector, uvector=uvector, svector=svector, cs=cs, dRuluu=dRuluu
    )
    #!ds^{mu}

    dy = np.zeros_like(y)

    dy[0:4] = uvector
    dy[4:8] = dp
    dy[8:] = ds

    return dy

In [175]:
t, r, theta, phi = smp.symbols("t r theta phi")
pt,pr,ptheta,pphi=sme.symbols("pt pr ptheta pphi")
st, sr, stheta, sphi = sme.symbols("st sr stheta sphi")
a,m,M= sme.symbols("a m M")

In [176]:
gk= cov_metric_tensor(r, theta,a ,M)

In [177]:
p=np.array([a,m,M])

In [178]:
y0=np.array([t,r,theta,phi,pt,pr,ptheta,pphi,st,sr,stheta,sphi])

In [179]:
f=MPD(0.,y0,p)

In [189]:
Vy0 = np.array(
    [
        137.46640187622472,
        5.84800301885202,
        0.9826545914525602,
        9.850782980810198,
        -0.9202703547946797,
        0.018850679031406487,
        -0.4361480888174733,
        2.2766562649705517,
        -0.028059659895012103,
        -1.0918957177510829,
        -2.5439915122039154,
        0.1878764770745049,
    ]
)

Vp = [1.0, 1.0, 1.0]

In [190]:
Vf = [
    f[i].subs(
        {
            t: Vy0[0],
            r: Vy0[1],
            theta: Vy0[2],
            phi: Vy0[3],
            pt: Vy0[4],
            pr: Vy0[5],
            ptheta: Vy0[6],
            pphi: Vy0[7],
            st: Vy0[8],
            sr: Vy0[9],
            stheta: Vy0[10],
            sphi: Vy0[11],
            a: Vp[0],
            M: Vp[2],
        }
    )
    for i in range(len(f))
]

In [191]:
np.array(Vf) - np.array(
    [
        1.3516279607171553,
        0.01233986051097109,
        -0.012478498506444897,
        0.10461590376658435,
        0.00010278560489796273,
        1.879940946991469e-05,
        0.1139839809737131,
        -0.005071480870330865,
        -0.029269696595498836,
        0.007869827633978104,
        -0.04881049574078713,
        0.4471133134487471,
    ]
)

array([0, 0, 0, 1.38777878078145e-17, -5.14996031930615e-19,
       -1.23599047663348e-17, -2.77555756156289e-17,
       -5.20417042793042e-18, 6.93889390390723e-18, 1.73472347597681e-18,
       0, -1.66533453693773e-16], dtype=object)

In [183]:
jacob = np.array([sme.Derivative(f[i], j) for i in range(len(f)) for j in y0])

In [184]:
jacobr = jacob.reshape((12, 12))

In [185]:
Vjacob = [
    jacob[i].subs(
        {
            t: Vy0[0],
            r: Vy0[1],
            theta: Vy0[2],
            phi: Vy0[3],
            pt: Vy0[4],
            pr: Vy0[5],
            ptheta: Vy0[6],
            pphi: Vy0[7],
            st: Vy0[8],
            sr: Vy0[9],
            stheta: Vy0[10],
            sphi: Vy0[11],
            a: Vp[0],
            M: Vp[2],
        }
    )
    for i in range(len(jacob))
]

Vjacobr = np.array(Vjacob).reshape((12, 12)).astype(float)

In [192]:
Njacob = np.array([[0.0, -0.09442309069089228, 0.014249577762831839, 0.0, -1.4793961791260521, -0.0008125092028435589, 0.00014655195342557162, -0.014824433663129546, 0.000664218042422268, 0.005253011537642131, 2.8853237455015845e-05, 2.69420679464842e-05], [0.0, 0.0012104159227546334, 0.00039636493612954987, 0.0, 0.00044351106949254855, 0.6744175580675817, 0.00025796733652435564, -3.159682057390807e-05, 0.009577793305550746, 0.0005447039501214106, 5.258558680417335e-05, 3.9265391884832396e-05], [0.0, 0.00415759355634497, -0.0003065574054613108, 0.0, -0.00020835787541671644, 0.00026632297109152733, 0.028609870087255452, 1.0791035175191658e-05, -0.0005221146728674492, -0.00028911044978544747, -3.16883010419708e-06, -0.00010511236188728859], [0.0, -0.036927504038478325, -0.12206878477201688, 0.0, -0.012330478209625453, -0.00010604381075705515, 1.523633615513058e-05, 0.040154756834841814, 9.402435247935883e-05, 0.0020847673448578918, 1.074875585839392e-05, -9.444456772300759e-06], [0.0, -8.38689512321203e-05, 0.00022635998605705836, 0.0, -6.700284327263947e-05, -0.00015816506164157307, -0.0002550079989995554, 1.4865954293776495e-05, -0.0007007345106629604, -0.00011814765220416904, 1.269144350320711e-06, -0.00013694108269431764], [0.0, -0.004333879221503046, -0.051673297324694854, 0.0, 0.09098941165025333, -0.0008196248378750131, -0.004374046464010564, 0.03596495558478916, 0.0001732207193947584, -0.0024849094267193742, 0.0010366693554489652, 0.0002777613901472551], [0.0, -0.03238339292227377, -0.45878340049714306, 0.0, 0.018727000553261347, -0.0007375578632789063, 0.00018448947868183823, 0.1068546048042098, -0.000755442993023316, 0.0266877968197137, 0.0018571976646866621, 6.271058417458775e-05], [0.0, 0.002503299870631774, -0.004253189256541038, 0.0, 0.0052014540369585815, -0.0009434658834071149, 0.0117549387487188, -5.33410868474534e-05, 0.00789315201830028, 0.004597730928345022, 7.702563051405685e-05, 0.0015658874370025045], [0.0, 0.01074495322635435, -0.0005434373625072034, 0.0, 0.030172710250389277, -0.0005217299560700556, -0.00021831716767910052, -0.0017115399884191059, 0.0006646932451814235, 0.023431237655704124, 0.001965274233824492, 0.00046677007678302593], [0.0, -0.004028822439799692, -0.0035069037162834887, 0.0, 0.0011713844128671018, 0.027541626327674113, -0.011902817994561402, 0.0013859683597278703, 0.04901042877250142, -5.5584258403737334e-05, -0.0021380574109480183, 0.01879396099647916], [0.0, 0.002356249135291137, -0.043200256333107555, 0.0, -0.002038532664652747, -0.2797071652335321, 0.12674735912209453, 0.004125963434729555, -0.012017386381837206, 0.04770232563111389, 0.0030102896021492833, 0.06108300316877156], [0.0, -0.08076983963733678, -0.14601916415081237, 0.0, -0.05092752846764018, 0.02169733442421828, 0.004133479455473205, 0.17600119516063287, -0.0007313848441872163, -0.2936868071507634, -0.04772958058346733, -0.007327235134684359]])

In [193]:
np.abs(Vjacobr - Njacob).max()

1.2012613126444194e-13

In [194]:
np.abs(Vjacobr - Njacob)

array([[0.00000000e+00, 7.63278329e-15, 1.49706636e-15, 0.00000000e+00,
        1.20126131e-13, 1.08637058e-16, 1.73201297e-17, 1.24032729e-15,
        5.42101086e-17, 4.30211422e-16, 2.81892565e-18, 2.16840434e-18],
       [0.00000000e+00, 9.84455573e-17, 3.19839641e-17, 0.00000000e+00,
        4.52654407e-17, 5.51780843e-14, 2.10877323e-17, 2.82570191e-18,
        7.85829735e-16, 4.45607093e-17, 4.30970364e-18, 3.13063377e-18],
       [0.00000000e+00, 3.40005801e-16, 2.49908601e-17, 0.00000000e+00,
        2.61834825e-17, 2.22261445e-17, 2.34187669e-15, 1.11300129e-18,
        4.21754645e-17, 2.37440276e-17, 2.64274280e-19, 8.57874969e-18],
       [0.00000000e+00, 2.99066327e-15, 9.99200722e-15, 0.00000000e+00,
        9.36750677e-16, 1.19397764e-17, 1.65679644e-18, 3.27515792e-15,
        7.68428290e-18, 1.70870262e-16, 9.89334482e-19, 7.72494048e-19],
       [0.00000000e+00, 6.88468380e-18, 1.00017650e-17, 0.00000000e+00,
        5.08897395e-17, 2.11885631e-15, 2.04372110e-17, 3.74

In [195]:
VEPS=np.finfo(float).eps

In [204]:
np.array(np.unravel_index(np.where((np.abs(Vjacobr - Njacob)>VEPS).flatten()),(12,12))).T

array([[[ 0,  1]],

       [[ 0,  2]],

       [[ 0,  4]],

       [[ 0,  7]],

       [[ 0,  9]],

       [[ 1,  5]],

       [[ 1,  8]],

       [[ 2,  1]],

       [[ 2,  6]],

       [[ 3,  1]],

       [[ 3,  2]],

       [[ 3,  4]],

       [[ 3,  7]],

       [[ 4,  5]],

       [[ 5,  1]],

       [[ 5,  2]],

       [[ 5,  4]],

       [[ 5,  7]],

       [[ 6,  1]],

       [[ 6,  2]],

       [[ 6,  4]],

       [[ 6,  5]],

       [[ 6,  7]],

       [[ 7,  1]],

       [[ 7,  2]],

       [[ 7,  4]],

       [[ 7,  5]],

       [[ 7,  6]],

       [[ 7,  7]],

       [[ 7,  8]],

       [[ 8,  1]],

       [[ 8,  4]],

       [[ 9,  1]],

       [[ 9,  2]],

       [[ 9,  5]],

       [[ 9,  6]],

       [[10,  1]],

       [[10,  2]],

       [[10,  5]],

       [[10,  6]],

       [[10,  7]],

       [[10,  8]],

       [[11,  1]],

       [[11,  2]],

       [[11,  4]],

       [[11,  5]],

       [[11,  6]],

       [[11,  7]],

       [[11,  9]]])

In [50]:
Vf[1]

0.00694117670375432

In [ ]:
M_ = 1e0
m_ = 1e0

S = 1e0 * m_ * M_  # 1e0 * m_ * M_
E = 0.9328 * m_
Jz = 2.8 * m_ * M_

r0 = 12.23 * M_
theta0 = np.pi / 7.0
phi0 = 0.0
a0 = 1.0 * M_

In [ ]:
# v_ = np.sqrt(M_ / r0)
# abar = a0 / M_

# Epro = (1 - 2 * v_**2 + abar * v_**3) / (np.sqrt(1 - 3 * v_**2 + 2 * abar * v_**3))
# Eret = (1 - 2 * v_**2 - abar * v_**3) / (np.sqrt(1 - 3 * v_**2 - 2 * abar * v_**3))

# ideg = np.pi / 30
# ecc = 0.01
# alpha_ip = 1 + np.cos(ideg)
# alpha_im = 1 - np.cos(ideg)

# E = 0.5 * (alpha_ip * Epro + alpha_im * Eret)
# Jz = np.cos(ideg) * np.sqrt((1 - ecc**2) / (2 * (1 - E)))

In [ ]:
gk = cov_metric_tensor(r=r0, theta=theta0, a=a0, M=M_)
gkinv = cot_metric_tensor(r=r0, theta=theta0, a=a0, M=M_)

In [ ]:
cs = kerr_christoffel(r=r0, theta=theta0, a=a0, M=M_)

In [ ]:
P1 = 1e-2 * m_

In [ ]:
S1 = 1e-4 * m_
S2 = 1e-4 * m_

In [ ]:
p0, p2, p3 = sme.symbols("p0 p2 p3", real=True)

In [ ]:
s0, s3 = sme.symbols("s0 s3", real=True)

In [ ]:
gpcoordu = np.array([p0, P1, p2, p3])
gscoordu = np.array([s0, S1, S2, s3])

In [ ]:
# Definitions
Delta = a0**2 - 2 * M_ * r0 + r0**2
Sigma = r0**2 + a0**2 * sme.cos(theta0) ** 2

In [ ]:
levi = levi_civita_tensor(gk)

In [ ]:
# Sab = spi(sa=gsi, pb=gpi, gk=eta, epsilon=epsl)
Suv = spintensor(levi=levi, pvector=gpcoordu, svector=gscoordu, m=m_)
#!Config: uu, In Coordinate Basis

In [ ]:
dgkt, dgkp = metric_D(r0, theta0, a0, M_)

In [ ]:
eq210a = energy(gpcoordu, Suv, dgkt) - E

In [ ]:
eq210b = angular_momentum(gpcoordu, Suv, dgkp) - Jz

In [ ]:
eq533 = get_norm(T=gpcoordu, g=gkinv) + m_**2

In [ ]:
eq229 = np.einsum("ij,j,i->", gkinv, gpcoordu, gscoordu) - 0.0

In [ ]:
eq228 = np.einsum("ij,j,i->", gkinv, gscoordu, gscoordu) - S**2

In [ ]:
def solve_eq(p_0, p_2, p_3, s_0, s_3):
    f = smp.lambdify([p0, p2, p3, s0, s3], [eq210a, eq210b, eq533, eq228, eq229])
    return f(p_0, p_2, p_3, s_0, s_3)

In [ ]:
def solve_eq_sp(p):
    f = sme.lambdify([p0, p2, p3, s0, s3], [eq210a, eq210b, eq533, eq228, eq229])
    return np.array(f(p[0], p[1], p[2], p[3], p[4]))

In [ ]:
def jac_nl_symb():
    X = smp.Matrix([eq210a, eq210b, eq533, eq228, eq229])
    Y = smp.Matrix([p0, p2, p3, s0, s3])

    return X.jacobian(Y)

In [ ]:
jac = jac_nl_symb()

In [ ]:
def jac_nl(p_0, p_2, p_3, s_0, s_3):
    f = smp.lambdify([p0, p2, p3, s0, s3], jac)

    return f(p_0, p_2, p_3, s_0, s_3)

In [ ]:
solve_eq(-0.924284, -1.33974, 2.55554, 0.0000381344, -0.000652199)

In [ ]:
roots = mpmath.findroot(
    solve_eq,
    (-0.92, -1.33, 2.55, 0.0000038, -0.00065),
    solver="mdnewton",
    tol=1e-19,
    J=jac_nl,
    verify=True,
)

In [ ]:
roots = np.array(roots.tolist(), dtype=np.float64)[:, 0]

In [ ]:
solve_eq(*roots)

In [ ]:
np0, np2, np3, ns0, ns3 = roots

In [ ]:
ngpcoordu = np.array([np0, P1, np2, np3]).astype(np.float64)
ngscoordu = np.array([ns0, S1, S2, ns3]).astype(np.float64)

In [ ]:
levi_mixed = np.einsum("px,sy,xyuv->psuv", gk, gk, levi)

In [ ]:
rulll = kerr_riemann_tensor(r=r0, theta=theta0, a=a0, M=M_, config="ulll")

In [ ]:
dRuluu = 0.5 * np.einsum("abps,psuv->abuv", rulll, levi)
ddRuuuu = 0.5 * np.einsum("bi,aips,psuv->abuv", gkinv, dRuluu, levi_mixed)

In [ ]:
dRlluu = np.einsum("ia,abuv->ibuv", gk, dRuluu)

In [ ]:
vmu0 = calculate_four_velocity(
    gk=gk, gkinv=gkinv, pvector=ngpcoordu, svector=ngscoordu, ddRuuuu=ddRuuuu
)

In [ ]:
#!We pass r0^mu,p^mu and S^mu
Vy0 = np.array(
    [
        0.0,
        r0,
        theta0,
        phi0,
        ngpcoordu[0],
        ngpcoordu[1],
        ngpcoordu[2],
        ngpcoordu[3],
        ngscoordu[0],
        ngscoordu[1],
        ngscoordu[2],
        ngscoordu[3],
    ]
).astype(np.float64)

In [72]:
gk = cov_metric_tensor(r, theta, a, M)  #! g_{ij}
gkinv = cot_metric_tensor(r, theta, a, M)
cs = kerr_christoffel(r, theta, a, M)  #! G^i_{jk}

Riemann = kerr_riemann_tensor(r, theta, a, M)  #! R^a_{bcd}

# Riemann_cov = np.einsum("ul,lvps->uvps", gk, Riemann)  #! R_{abcd}
# Riemann_uull = np.einsum("ap,upgd->uagd", gkinv, Riemann)  #! R^{ua}_{gd}

levi = levi_civita_tensor(gk)  #! e^{abcd}
# levi_mixed = np.einsum("psxy,xu,yv->psuv", levi, gk, gk)  #! e^{ps}_{uv}
levi_mixed = np.einsum("px,sy,xyuv->psuv", gk, gk, levi)  #! e_{ps}^{uv}

dRuluu = 0.5 * np.einsum("abps,psuv->abuv", Riemann, levi) #!R*^a_{buv}
dRlluu = np.einsum("ia,abuv->ibuv", gk, dRuluu)  #!R*^_{ab}^{uv}

ddRuuuu = 0.5 * np.einsum("bi,aips,psuv->abuv", gkinv, dRuluu, levi_mixed) #! *R*^{abuv}


In [73]:
DdRuluu= np.zeros((4,4,4,4,4),object)
VDdRuluu= np.zeros((4,4,4,4,4))

In [74]:
vari=[t,r,theta,phi]

In [75]:
for x_ in range(4):
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    if dRuluu[i,j,k,l]!=0:
                        DdRuluu[i,j,k,l,x_]= dRuluu[i,j,k,l].diff(vari[x_])
                        VDdRuluu[i,j,k,l,x_]= DdRuluu[i,j,k,l,x_].subs(
                                                {
                                                    t: Vy0[0],
                                                    r: Vy0[1],
                                                    theta: Vy0[2],
                                                    phi: Vy0[3],
                                                    pt: Vy0[4],
                                                    pr: Vy0[5],
                                                    ptheta: Vy0[6],
                                                    pphi: Vy0[7],
                                                    st: Vy0[8],
                                                    sr: Vy0[9],
                                                    stheta: Vy0[10],
                                                    sphi: Vy0[11],
                                                    a: Vp[0],
                                                    M: Vp[2],
                                                }
                                            ) 

In [76]:
(VDdRuluu-np.array([[[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 2.4984307418550448e-05, 2.755797206243048e-05, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -3.875151999707456e-05, 3.4208518133781196e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 3.875151999707456e-05, -3.4208518133781196e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -2.4984307418550448e-05, -2.755797206243048e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.003238443135282057, 0.005839912233519245, 0.0], [0.0, 0.00046868477249332823, -0.0004397261424367534, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.003238443135282057, -0.005839912233519245, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.0001715818735866389, -0.0002212492432959475, 0.0]], [[0.0, -0.00046868477249332823, 0.0004397261424367534, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.0004574625804558327, -0.00023134595223956838, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0001715818735866389, 0.0002212492432959475, 0.0], [0.0, 0.0004574625804558327, 0.00023134595223956838, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0060411986649071435, -0.010334942128345053, 0.0], [0.0, -0.001661310655092037, -0.0029220246326784006, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.0060411986649071435, 0.010334942128345053, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.003099568849400926, -0.002571870628701292, 0.0]], [[0.0, 0.001661310655092037, 0.0029220246326784006, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.00014659756616808848, 0.00019369127123351702, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.003099568849400926, 0.002571870628701292, 0.0], [0.0, -0.00014659756616808848, -0.00019369127123351702, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.0015771324801900194, -0.002917887600840845, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0020405558865640343, -0.0035043893885120746, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -0.0020405558865640343, 0.0035043893885120746, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0015771324801900194, 0.002917887600840845, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]]], [[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0011824917946733544, 0.0026044052465769457, 0.0], [0.0, 0.0002160077730581736, -0.00024043464988412238, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.0011824917946733544, -0.0026044052465769457, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -7.681312691094639e-05, -0.00011096738008435571, 0.0]], [[0.0, -0.0002160077730581736, 0.00024043464988412238, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.0001746019576498942, -9.817117115015213e-05, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 7.681312691094639e-05, 0.00011096738008435571, 0.0], [0.0, 0.0001746019576498942, 9.817117115015213e-05, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0028141130893991418, 0.0028654841184450213, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0005646616708371752, 0.0013015510237118174, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -0.0005646616708371752, -0.0013015510237118174, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.0028141130893991418, -0.0028654841184450213, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, -0.0012093433850276696, -4.5871488544101504e-05, 0.0], [0.0, -0.0021632489878432586, 0.00386253474949485, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0012093433850276696, 4.5871488544101504e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0006178301238361791, 0.0013028542228651278, 0.0]], [[0.0, 0.0021632489878432586, -0.00386253474949485, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0002160077730581736, -0.00024043464988412238, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -0.0006178301238361791, -0.0013028542228651278, 0.0], [0.0, -0.0002160077730581736, 0.00024043464988412238, 0.0], [0.0, 0.0, 0.0, 0.0]]]], [[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.00025475929305524816, -0.0002746431680179035, 0.0], [0.0, -4.379363537233184e-05, -5.5433214280577314e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.00025475929305524816, 0.0002746431680179035, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -9.311959660810034e-05, -4.4698380486838554e-05, 0.0]], [[0.0, 4.379363537233184e-05, 5.5433214280577314e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 4.703813975754476e-06, 4.72138666814387e-06, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 9.311959660810034e-05, 4.4698380486838554e-05, 0.0], [0.0, -4.703813975754476e-06, -4.72138666814387e-06, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.000219652153565718, -0.00012191923882784048, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -4.0537752916512673e-05, -5.537776639036219e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 4.0537752916512673e-05, 5.537776639036219e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.000219652153565718, 0.00012191923882784048, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, -0.004203804874407294, 0.007366924138006925, 0.0], [0.0, 8.64318291963315e-05, 1.9517180120445776e-06, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.004203804874407294, -0.007366924138006925, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.00025475929305524816, -0.0002746431680179035, 0.0]], [[0.0, -8.64318291963315e-05, -1.9517180120445776e-06, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -8.433138828884451e-05, -0.0001108109806709395, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -0.00025475929305524816, 0.0002746431680179035, 0.0], [0.0, 8.433138828884451e-05, 0.0001108109806709395, 0.0], [0.0, 0.0, 0.0, 0.0]]]], [[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -5.6138037571692235e-05, -0.00015477544964522067, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 8.148236104179384e-05, 5.347279066331354e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -8.148236104179384e-05, -5.347279066331354e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 5.6138037571692235e-05, 0.00015477544964522067, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0001715818735866389, 0.0002212492432959475, 0.0], [0.0, 0.00023781042689011454, 0.00010942671341172785, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.0001715818735866389, -0.0002212492432959475, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -6.342011595243284e-05, -0.000161844009281799, 0.0]], [[0.0, -0.00023781042689011454, -0.00010942671341172785, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -2.6233483855109443e-05, -1.0036188327538958e-05, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 6.342011595243284e-05, 0.000161844009281799, 0.0], [0.0, 2.6233483855109443e-05, 1.0036188327538958e-05, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.005913681938800069, 0.005437354747146314, 0.0], [0.0, -0.00014659756616808848, -0.00019369127123351702, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.005913681938800069, -0.005437354747146314, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.000412401495316937, -0.00023588187179297848, 0.0]], [[0.0, 0.00014659756616808848, 0.00019369127123351702, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.00011955815352412505, 0.00031661945892701973, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.000412401495316937, 0.00023588187179297848, 0.0], [0.0, -0.00011955815352412505, -0.00031661945892701973, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -2.4984307418550448e-05, -2.755797206243048e-05, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 3.875151999707457e-05, -3.4208518133781196e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -3.875151999707457e-05, 3.4208518133781196e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 2.4984307418550448e-05, 2.755797206243048e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]]]])).max()

6.938893903907228e-18

In [78]:
DddRuuuu = np.zeros((4, 4, 4, 4, 4), object)
VDddRuuuu = np.zeros((4, 4, 4, 4, 4))

In [79]:
for x_ in range(4):
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    if dRuluu[i, j, k, l] != 0:
                        DddRuuuu[i, j, k, l, x_] = ddRuuuu[i, j, k, l].diff(vari[x_])
                        VDddRuuuu[i, j, k, l, x_] = DddRuuuu[i, j, k, l, x_].subs(
                            {
                                t: Vy0[0],
                                r: Vy0[1],
                                theta: Vy0[2],
                                phi: Vy0[3],
                                pt: Vy0[4],
                                pr: Vy0[5],
                                ptheta: Vy0[6],
                                pphi: Vy0[7],
                                st: Vy0[8],
                                sr: Vy0[9],
                                stheta: Vy0[10],
                                sphi: Vy0[11],
                                a: Vp[0],
                                M: Vp[2],
                            }
                        )

In [82]:
np.abs(
    VDddRuuuu
    - np.array(
        [
            [
                [
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -4.1612266815430934e-20, 4.342975228557504e-21, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -1.8077543720684546e-21, -7.530126344887785e-21, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 1.8077543720684546e-21, 7.530126344887785e-21, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 4.1612266815430934e-20, -4.342975228557504e-21, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -0.005052813395808382, 0.0022390494642794516, 0.0],
                            [0.0, 0.00010388776773303986, 7.03052297260404e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.005052813395808382, -0.0022390494642794516, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0003062109700887875, -8.10455985625786e-05, 0.0],
                        ],
                        [
                            [0.0, -0.00010388776773303986, -7.03052297260404e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [
                                0.0,
                                -0.00010136311773820613,
                                -0.00019928861916706207,
                                0.0,
                            ],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -0.0003062109700887875, 8.10455985625786e-05, 0.0],
                            [0.0, 0.00010136311773820613, 0.00019928861916706207, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.00010388776773303986, 7.030522972604038e-05, 0.0],
                            [0.0, 0.00019939922077450345, -5.403695066336312e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, -0.00010388776773303986, -7.030522972604038e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -5.263828223988749e-05, -0.00010064298783401111, 0.0],
                        ],
                        [
                            [0.0, -0.00019939922077450345, 5.403695066336312e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -1.6785830672218062e-05, 3.0187832138628413e-06, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 5.263828223988749e-05, 0.00010064298783401111, 0.0],
                            [0.0, 1.6785830672218062e-05, -3.0187832138628413e-06, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0002640135251547569, 0.0003406799004403646, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 4.8724835498318645e-05, 9.864563133305096e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -4.8724835498318645e-05, -9.864563133305096e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, -0.0002640135251547569, -0.0003406799004403646, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                ],
                [
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.005052813395808381, -0.002239049464279448, 0.0],
                            [0.0, -0.00010388776773303988, -7.030522972604049e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, -0.005052813395808381, 0.002239049464279448, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -0.0003062109700887875, 8.104559856257862e-05, 0.0],
                        ],
                        [
                            [0.0, 0.00010388776773303988, 7.030522972604049e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.00010136311773820613, 0.00019928861916706207, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0003062109700887875, -8.104559856257862e-05, 0.0],
                            [
                                0.0,
                                -0.00010136311773820613,
                                -0.00019928861916706207,
                                0.0,
                            ],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 4.872483549831866e-05, 9.864563133305096e-05, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -7.101842233532942e-05, 2.0285376606518608e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 7.101842233532942e-05, -2.0285376606518608e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, -4.872483549831866e-05, -9.864563133305096e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0003062109700887875, -8.10455985625787e-05, 0.0],
                            [
                                0.0,
                                -5.2638282239887514e-05,
                                -0.00010064298783401111,
                                0.0,
                            ],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, -0.0003062109700887875, 8.10455985625787e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [
                                0.0,
                                -0.00011192620951989865,
                                -0.00014909386369877997,
                                0.0,
                            ],
                        ],
                        [
                            [0.0, 5.2638282239887514e-05, 0.00010064298783401111, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 5.65380529738168e-06, 8.464248170640558e-06, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.00011192620951989865, 0.00014909386369877997, 0.0],
                            [0.0, -5.65380529738168e-06, -8.464248170640558e-06, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                ],
                [
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -0.00010388776773303986, -7.030522972604049e-05, 0.0],
                            [0.0, -0.00019939922077450345, 5.403695066336306e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.00010388776773303986, 7.030522972604049e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 5.263828223988749e-05, 0.00010064298783401114, 0.0],
                        ],
                        [
                            [0.0, 0.00019939922077450345, -5.403695066336306e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 1.678583067221806e-05, -3.0187832138628367e-06, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -5.263828223988749e-05, -0.00010064298783401114, 0.0],
                            [0.0, -1.678583067221806e-05, 3.0187832138628367e-06, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -4.8724835498318645e-05, -9.864563133305096e-05, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 7.101842233532945e-05, -2.0285376606518608e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -7.101842233532945e-05, 2.0285376606518608e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 4.8724835498318645e-05, 9.864563133305096e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [
                                0.0,
                                -0.00010136311773820616,
                                -0.00019928861916706212,
                                0.0,
                            ],
                            [0.0, -1.6785830672218062e-05, 3.01878321386284e-06, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.00010136311773820616, 0.00019928861916706212, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 5.653805297381679e-06, 8.46424817064056e-06, 0.0],
                        ],
                        [
                            [0.0, 1.6785830672218062e-05, -3.01878321386284e-06, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 1.3740149267288661e-05, 1.2797315202509238e-05, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -5.653805297381679e-06, -8.46424817064056e-06, 0.0],
                            [
                                0.0,
                                -1.3740149267288661e-05,
                                -1.2797315202509238e-05,
                                0.0,
                            ],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                ],
                [
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -0.0002640135251547567, -0.0003406799004403644, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -4.872483549831863e-05, -9.864563133305096e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 4.872483549831863e-05, 9.864563133305096e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0002640135251547567, 0.0003406799004403644, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -0.0003062109700887874, 8.104559856257881e-05, 0.0],
                            [0.0, 5.263828223988753e-05, 0.00010064298783401114, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0003062109700887874, -8.104559856257881e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.00011192620951989862, 0.00014909386369877994, 0.0],
                        ],
                        [
                            [0.0, -5.263828223988753e-05, -0.00010064298783401114, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -5.653805297381679e-06, -8.464248170640555e-06, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [
                                0.0,
                                -0.00011192620951989862,
                                -0.00014909386369877994,
                                0.0,
                            ],
                            [0.0, 5.653805297381679e-06, 8.464248170640555e-06, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0001013631177382061, 0.00019928861916706212, 0.0],
                            [0.0, 1.6785830672218062e-05, -3.018783213862845e-06, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, -0.0001013631177382061, -0.00019928861916706212, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -5.653805297381681e-06, -8.464248170640558e-06, 0.0],
                        ],
                        [
                            [0.0, -1.6785830672218062e-05, 3.018783213862845e-06, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -1.3740149267288658e-05, -1.279731520250924e-05, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 5.653805297381681e-06, 8.464248170640558e-06, 0.0],
                            [0.0, 1.3740149267288658e-05, 1.279731520250924e-05, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                    [
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 1.8482260277343572e-21, 3.1470064661099616e-21, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 1.2602404780058114e-22, -1.6549006505086217e-22, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, -1.2602404780058114e-22, 1.6549006505086217e-22, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                        [
                            [
                                0.0,
                                -1.8482260277343572e-21,
                                -3.1470064661099616e-21,
                                0.0,
                            ],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0],
                        ],
                    ],
                ],
            ]
        ]
    )
).max()

9.540979117872439e-18

In [77]:
# !jupytext --output MPDJacobian.py MPDJacobian.ipynb

In [83]:
1.0 + (-8.88178420e-20)

1.0